In [1]:
import torch
import torch.nn as nn

In [2]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Block, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        return x

In [3]:
class ResNet1D(nn.Module):
    def __init__(self, Block, in_channels, num_classes):
        super(ResNet1D, self).__init__()
        self.in_channels = 2048
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=2048, kernel_size=7, stride=2, bias=False)
        self.bn1 = nn.BatchNorm1d(2048)
    
        self.layer1 = self._make_layer(Block, out_channels=1024)
        self.layer2 = self._make_layer(Block, out_channels=512)
        self.layer3 = self._make_layer(Block, out_channels=256)
        self.layer4 = self._make_layer(Block, out_channels=64)
        
        self.fc1 = nn.Linear(in_features=160, out_features=512, bias=False)
        self.bn2 = nn.BatchNorm1d(6)
        self.fc2 = nn.Linear(in_features=512, out_features=num_classes, bias=False)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        b_size = x.size(0)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = x.reshape(b_size, 6, -1)
        
        x = self.relu(self.bn2(self.fc1(x)))
        x = self.fc2(x)
        return x
    
    def _make_layer(self, Block, out_channels):
        layers = []
        for i in range(2):
            block = Block(self.in_channels, out_channels)
            layers.append(block)
            self.in_channels = out_channels
        return nn.Sequential(*layers)
    

In [4]:
model = ResNet1D(Block, in_channels=6044, num_classes=9)

x = torch.randn((1, 6044, 99))
model(x).shape

torch.Size([1, 6, 9])